# KuCoin Updater

In [ ]:
import pandas as pd
import numpy as np


import os
import requests
import json
import base64

import time
import base64
import hmac
import hashlib

## Run this to append KuCoin Data file (holdings)

In [ ]:
response = requests.request('get', url, headers=headers)

In [2]:
def kucoin_append():
    
    # First importing the needed libraries:

    import pandas as pd
    import numpy as np


    import os
    import requests
    import json
    import base64
    import time
    import base64
    import hmac
    import hashlib

    # Now importing the env file so the script can access the KuCoin API keys:
    import env

    # Defining the api keys with their own variables:
    api_key = env.kc_api_key
    api_s = env.kc_secret_api
    api_pp = env.kc_passphrase
    api_uid = env.kc_uid

    # creating the api keys for use in the calls:
    api_key = env.kc_api_key
    api_secret = env.kc_secret_api
    api_passphrase = env.kc_passphrase
    url = 'https://api.kucoin.com/api/v1/accounts'
    now = int(time.time() * 1000)
    str_to_sign = str(now) + 'GET' + '/api/v1/accounts'
    signature = base64.b64encode(
    hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
    "KC-API-SIGN": signature,
    "KC-API-TIMESTAMP": str(now),
    "KC-API-KEY": api_key,
    "KC-API-PASSPHRASE": passphrase,
    "KC-API-KEY-VERSION": str(2)
    }

    # Getting the base response with the top level account values:
    response = requests.request('get', url, headers=headers)


    # Creating the account dataframe using the response request I just created:
    df = pd.DataFrame.from_dict(response.json()['data'])

    # Column cleanup:
    df.drop(columns = 'id', inplace = True)

    # Getting prices for coins:
    coin_list = df['currency'].unique().tolist()

    # USDC and USDT don't work in this list because they are "equivalent" of USD, so it comes back as a NoneType, leading to a none-type error later if I don't remove them from the list at this point.
    coin_list.remove('USDC')
    coin_list.remove('USDT')
    
    # This for loop will create a list of prices by calling each crypto within my 'coin_list' list. 

    price_list = []
    for coin in coin_list:
        prices = float(requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={coin}-USDT').json()['data']['price'])
    #     print(prices), print(type(prices))
        price_list.append(prices)

    # Now creating a dictionary of the coin prices:
    coin_dict = {"coin":coin_list, "price":price_list}

    # Dataframe from the dictionary:
    df_prices = pd.DataFrame(coin_dict)

    # creating a copy dataframe of the acct info (this step can be dropped in future)
    account = df[df['type'] == 'trade'].copy()
    account = account.reset_index(drop = True)
    account.rename(columns = {'type': "act_name"}, inplace = True)


    # Now should be able to join the two dataframes. I have to join the two dataframes first before I can multiply columns to create the "$ value" column which is the end goal here.

    # Merge, inner join:

    holdings_append = account.merge(df_prices, left_on = 'currency', right_on = 'coin', how = 'left')

    # Now changing the value types of the columns with numbers in them from objects to float64:
    holdings_append['balance'] = holdings_append.balance.astype(float)
    holdings_append['available'] = holdings_append.available.astype(float)
    holdings_append['holds'] = holdings_append.holds.astype(float)

    # Now working through adding calculated columns that I'll later select from if it doesn't return 'nan':

    holdings_append['value_tmp'] = round(holdings_append.price * holdings_append.balance, 2)

    holdings_append["dollar_value"] = np.where(holdings_append['value_tmp'].notnull(), holdings_append['value_tmp'], holdings_append['balance'])
    holdings_append.dollar_value = holdings_append.dollar_value.round(2)

    # Dropping extra columns:
    holdings_append.drop(columns = ['coin', 'value_tmp'], inplace = True)

    # Adding date column, and changing date to the dataframe index:
    # holdings.insert(0, 'date', pd.to_datetime('today').strftime('%Y-%m-%d'))
    holdings_append.insert(0, 'date', pd.to_datetime('now').replace(microsecond=0))
    holdings_append.date = pd.to_datetime(holdings_append.date)
    holdings_append = holdings_append.set_index('date').sort_index()

    # printing out result, and saving to csv and Excel.
    return holdings_append

    # holdings.to_csv("account_holdings_index_test.csv")
    # holdings.to_csv("account_holdings_no_index.csv", index = False)


    # holdings.to_excel("account_holdings_index.xlsx")
    # holdings.to_excel("account_holdings_no_index.xlsx", index = False)
    # print("CSV written successfully.")

In [3]:
kucoin_append()

Keys loaded successfully


,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-12 19:08:11,USDC,trade,1879.263101,1879.263101,0.00000,NaN,1879.26
2021-09-12 19:08:11,USDT,trade,1793.670859,1334.388009,459.28285,NaN,1793.67
2021-09-12 19:08:11,BTC3L,trade,918.961600,488.156400,430.80520,0.7877,723.87
2021-09-12 19:08:11,BTC,trade,0.013654,0.013654,0.00000,45877.8000,626.43
2021-09-12 19:08:11,BTC3S,trade,225.609100,166.834800,58.77430,2.2630,510.55
2021-09-12 19:08:11,ATOM,trade,0.236600,0.236600,0.00000,32.8393,7.77
2021-09-12 19:08:11,ALGO,trade,0.000000,0.000000,0.00000,2.3691,0.00
2021-09-12 19:08:11,ETH,trade,0.000000,0.000000,0.00000,3407.5000,0.00
2021-09-12 19:08:11,MATIC,trade,0.000000,0.000000,0.00000,1.3366,0.00


In [44]:
# First importing the needed libraries:

import pandas as pd
import numpy as np


import os
import requests
import json
import base64
import time
import base64
import hmac
import hashlib

# Now importing the env file so the script can access the KuCoin API keys:
import env

# Defining the api keys with their own variables:
api_key = env.kc_api_key
api_s = env.kc_secret_api
api_pp = env.kc_passphrase
api_uid = env.kc_uid

# creating the api keys for use in the calls:
api_key = env.kc_api_key
api_secret = env.kc_secret_api
api_passphrase = env.kc_passphrase
url = 'https://api.kucoin.com/api/v1/accounts'
now = int(time.time() * 1000)
str_to_sign = str(now) + 'GET' + '/api/v1/accounts'
signature = base64.b64encode(
hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
headers = {
"KC-API-SIGN": signature,
"KC-API-TIMESTAMP": str(now),
"KC-API-KEY": api_key,
"KC-API-PASSPHRASE": passphrase,
"KC-API-KEY-VERSION": str(2)
}

# Getting the base response with the top level account values:
response = requests.request('get', url, headers=headers)


# Creating the account dataframe using the response request I just created:
df = pd.DataFrame.from_dict(response.json()['data'])

# Column cleanup:
df.drop(columns = 'id', inplace = True)

# Getting prices for coins:

coin_list = ['BTC', 'ETH', 'ADA', 'ALGO', 'ATOM', 'XLM', 'DOT']

# This for loop will create a list of prices by calling each crypto within my 'coin_list' list. 

price_list = []
for coin in coin_list:
    prices = float(requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={coin}-USDT').json()['data']['price'])
#     print(prices), print(type(prices))
    price_list.append(prices)

# Now creating a dictionary of the coin prices:
coin_dict = {"coin":coin_list, "price":price_list}

# Dataframe from the dictionary:
df_prices = pd.DataFrame(coin_dict)

# creating a copy dataframe of the acct info (this step can be dropped in future)
account = df[df['type'] == 'trade'].copy()
account = account.reset_index(drop = True)
account.rename(columns = {'type': "act_name"}, inplace = True)


# Now should be able to join the two dataframes. I have to join the two dataframes first before I can multiply columns to create the "$ value" column which is the end goal here.

# Merge, inner join:

holdings_append = account.merge(df_prices, left_on = 'currency', right_on = 'coin', how = 'left')

# Now changing the value types of the columns with numbers in them from objects to float64:
holdings_append['balance'] = holdings_append.balance.astype(float)
holdings_append['available'] = holdings_append.available.astype(float)
holdings_append['holds'] = holdings_append.holds.astype(float)

# Now working through adding calculated columns that I'll later select from if it doesn't return 'nan':

holdings_append['value_tmp'] = round(holdings_append.price * holdings_append.balance, 2)

holdings_append["dollar_value"] = np.where(holdings_append['value_tmp'].notnull(), holdings_append['value_tmp'], holdings_append['balance'])
holdings_append.dollar_value = holdings_append.dollar_value.round(2)

# Dropping extra columns:
holdings_append.drop(columns = ['coin', 'value_tmp'], inplace = True)

# Adding date column, and changing date to the dataframe index:
# holdings.insert(0, 'date', pd.to_datetime('today').strftime('%Y-%m-%d'))
holdings_append.insert(0, 'date', pd.to_datetime('now').replace(microsecond=0))
holdings_append.date = pd.to_datetime(holdings_append.date)
holdings_append = holdings_append.set_index('date').sort_index()

# printing out result, and saving to csv and Excel.
holdings_append

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-03 15:59:56,USDT,trade,2033.431652,1683.469627,349.962025,NaN,2033.43
2021-09-03 15:59:56,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-03 15:59:56,BTC,trade,0.013654,0.013654,0.000000,50567.6000,690.47
2021-09-03 15:59:56,BTC3S,trade,342.742100,120.711700,222.030400,NaN,342.74
2021-09-03 15:59:56,BTC3L,trade,343.540200,236.594900,106.945300,NaN,343.54
2021-09-03 15:59:56,ATOM,trade,0.236600,0.236600,0.000000,24.2136,5.73
2021-09-03 15:59:56,ALGO,trade,0.000000,0.000000,0.000000,1.2355,0.00
2021-09-03 15:59:56,ETH,trade,0.000000,0.000000,0.000000,3969.2000,0.00
2021-09-03 15:59:56,MATIC,trade,0.000000,0.000000,0.000000,NaN,0.00


In [45]:
holdings_append

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-03 15:59:56,USDT,trade,2033.431652,1683.469627,349.962025,NaN,2033.43
2021-09-03 15:59:56,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-03 15:59:56,BTC,trade,0.013654,0.013654,0.000000,50567.6000,690.47
2021-09-03 15:59:56,BTC3S,trade,342.742100,120.711700,222.030400,NaN,342.74
2021-09-03 15:59:56,BTC3L,trade,343.540200,236.594900,106.945300,NaN,343.54
2021-09-03 15:59:56,ATOM,trade,0.236600,0.236600,0.000000,24.2136,5.73
2021-09-03 15:59:56,ALGO,trade,0.000000,0.000000,0.000000,1.2355,0.00
2021-09-03 15:59:56,ETH,trade,0.000000,0.000000,0.000000,3969.2000,0.00
2021-09-03 15:59:56,MATIC,trade,0.000000,0.000000,0.000000,NaN,0.00


In [3]:
# THIS CODE works to append to the csv file itself:

holdings_append.to_csv('data.csv', mode = 'a', header = False)

In [4]:
holdings_append.shape

(9, 7)

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(credentials)

spreadsheet = client.open('Crypto-Data')

with open('data.csv', 'r') as file_obj:
    content = file_obj.read()
    client.import_csv(spreadsheet.id, data=content)

## Full Function start to finish

In [44]:
def kucoin_append():
    
    # First importing the needed libraries:

    import pandas as pd
    import numpy as np


    import os
    import requests
    import json
    import base64
    import time
    import base64
    import hmac
    import hashlib

    # Now importing the env file so the script can access the KuCoin API keys:
    import env

    # Defining the api keys with their own variables:
    api_key = env.kc_api_key
    api_s = env.kc_secret_api
    api_pp = env.kc_passphrase
    api_uid = env.kc_uid

    # creating the api keys for use in the calls:
    api_key = env.kc_api_key
    api_secret = env.kc_secret_api
    api_passphrase = env.kc_passphrase
    url = 'https://api.kucoin.com/api/v1/accounts'
    now = int(time.time() * 1000)
    str_to_sign = str(now) + 'GET' + '/api/v1/accounts'
    signature = base64.b64encode(
    hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
    "KC-API-SIGN": signature,
    "KC-API-TIMESTAMP": str(now),
    "KC-API-KEY": api_key,
    "KC-API-PASSPHRASE": passphrase,
    "KC-API-KEY-VERSION": str(2)
    }

    # Getting the base response with the top level account values:
    response = requests.request('get', url, headers=headers)


    # Creating the account dataframe using the response request I just created:
    df = pd.DataFrame.from_dict(response.json()['data'])

    # Column cleanup:
    df.drop(columns = 'id', inplace = True)

    # Getting prices for coins:
    coin_list = df['currency'].unique().tolist()

    # USDC and USDT don't work in this list because they are "equivalent" of USD, so it comes back as a NoneType, leading to a none-type error later if I don't remove them from the list at this point.
    coin_list.remove('USDC')
    coin_list.remove('USDT')
    
    # This for loop will create a list of prices by calling each crypto within my 'coin_list' list. 

    price_list = []
    for coin in coin_list:
        prices = float(requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={coin}-USDT').json()['data']['price'])
    #     print(prices), print(type(prices))
        price_list.append(prices)

    # Now creating a dictionary of the coin prices:
    coin_dict = {"coin":coin_list, "price":price_list}

    # Dataframe from the dictionary:
    df_prices = pd.DataFrame(coin_dict)

    # creating a copy dataframe of the acct info (this step can be dropped in future)
    account = df[df['type'] == 'trade'].copy()
    account = account.reset_index(drop = True)
    account.rename(columns = {'type': "act_name"}, inplace = True)


    # Now should be able to join the two dataframes. I have to join the two dataframes first before I can multiply columns to create the "$ value" column which is the end goal here.

    # Merge, inner join:

    holdings_append = account.merge(df_prices, left_on = 'currency', right_on = 'coin', how = 'left')

    # Now changing the value types of the columns with numbers in them from objects to float64:
    holdings_append['balance'] = holdings_append.balance.astype(float)
    holdings_append['available'] = holdings_append.available.astype(float)
    holdings_append['holds'] = holdings_append.holds.astype(float)

    # Now working through adding calculated columns that I'll later select from if it doesn't return 'nan':

    holdings_append['value_tmp'] = round(holdings_append.price * holdings_append.balance, 2)

    holdings_append["dollar_value"] = np.where(holdings_append['value_tmp'].notnull(), holdings_append['value_tmp'], holdings_append['balance'])
    holdings_append.dollar_value = holdings_append.dollar_value.round(2)

    # Dropping extra columns:
    holdings_append.drop(columns = ['coin', 'value_tmp'], inplace = True)

    # Adding date column, and changing date to the dataframe index:
    # holdings.insert(0, 'date', pd.to_datetime('today').strftime('%Y-%m-%d'))
    holdings_append.insert(0, 'date', pd.to_datetime('now').replace(microsecond=0))
    holdings_append.date = pd.to_datetime(holdings_append.date)
    holdings_append = holdings_append.set_index('date').sort_index()
    
        
    # Calcuating the total portfolio value in $:
    total_port = holdings_append.dollar_value.sum(axis = 0, skipna = True)
    
    # adding each holding's percentage of the overall portfolio:
    holdings_append['percentages'] = round(holdings_append.dollar_value / total_port, 2)

    # printing out result, and saving to csv and Excel.
    return holdings_append

    # holdings.to_csv("account_holdings_index_test.csv")
    # holdings.to_csv("account_holdings_no_index.csv", index = False)


    # holdings.to_excel("account_holdings_index.xlsx")
    # holdings.to_excel("account_holdings_no_index.xlsx", index = False)
    # print("CSV written successfully.")
    
holdings_append = kucoin_append()   


# THIS CODE works to append to the csv file itself. Only works if there is an existing file. I will add the if statement later that'll check if a file exists and create a new one, or use the append function:
holdings_append.to_csv('data.csv', mode = 'a', header = False)

# Uploading to Google sheets

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(credentials)

spreadsheet = client.open('Crypto-Data')

with open('data.csv', 'r') as file_obj:
    content = file_obj.read()
    client.import_csv(spreadsheet.id, data=content)
    

# Ending script    
import time

t = int(5)

def countdown(t):
    print(f"Script will exit in {t} seconds")
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        print(timer, end = "\r")
        time.sleep(1)
        t -= 1
    print("Limitless Holdings database and Google Sheets have been updated. Have a nice day!")
    
countdown(t)

Script will exit in 5 seconds
Limitless Holdings database and Google Sheets have been updated. Have a nice day!


In [12]:
kucoin_append()

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-04 22:59:19,USDT,trade,2034.507513,1687.675475,346.832038,NaN,2034.51
2021-09-04 22:59:19,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26
2021-09-04 22:59:19,BTC,trade,0.013654,0.013654,0.000000,49871.9000,680.97
2021-09-04 22:59:19,BTC3S,trade,317.191700,120.711700,196.480000,1.9058,604.50
2021-09-04 22:59:19,BTC3L,trade,382.546100,236.594900,145.951200,1.2662,484.38
2021-09-04 22:59:19,ATOM,trade,0.236600,0.236600,0.000000,24.2881,5.75
2021-09-04 22:59:19,ALGO,trade,0.000000,0.000000,0.000000,1.2235,0.00
2021-09-04 22:59:19,ETH,trade,0.000000,0.000000,0.000000,3886.9200,0.00
2021-09-04 22:59:19,MATIC,trade,0.000000,0.000000,0.000000,1.6446,0.00


## Writing code to SQLite db

In [9]:
import sqlite3 as sql

### Creating the Database
- This step will only need to be run once. 

In [ ]:
# dataframe that will create the new database:

holdings_append.head()

In [10]:
# conn = sql.connect('kucoin.db')

conn = sql.connect('/Users/luke/projects/github/algo-trading-experiments/kucoin.db')

In [ ]:
# Only need to run this once
holdings_append.to_sql('holdings', con = conn)

In [44]:
import pandas as pd

holdings_test = pd.read_sql("SELECT * FROM sqlite_master WHERE type = 'table' AND name = 'holdings';", conn)

In [45]:
holdings_test.name.count()


1

### Appending the SQLite database

In [46]:
import sqlite3 as sql
import pandas as pd
import time

# This line must be run first to setup the connection.
conn = sql.connect('/Users/luke/projects/github/algo-trading-experiments/kucoin.db')

# This code checks to see if the SQLite database has a table named "holdings".
holdings_test = pd.read_sql("SELECT * FROM sqlite_master WHERE type = 'table' AND name = 'holdings';", conn)

# Now the code checks if the table exists, and if so appends the newest holdings dataframe to the sqlite database.
if holdings_test.name.count() > 0:
    holdings_append.to_sql('holdings', con = conn, if_exists = 'append')
else:
    # This line creates a table called "holdings" if the table doesn't exist.
    holdings_append.to_sql('holdings', con = conn)
    
print("SQLite database has been updated.")

SQLite database has been updated.


In [54]:
import time

t = int(5)

def countdown(t):
    print(f"Script will exit in {t} seconds")
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        print(timer, end = "\r")
        time.sleep(1)
        t -= 1
    print("Limitless Holdings database and Google Sheets have been updated. Have a nice day!")



In [55]:
countdown(t)

Script will exit in 5 seconds
Limitless Holdings database and Google Sheet has been updated. Thank you.


In [48]:
print("This is the start of the timer...")

time.sleep(3)
print("Printed after 3 seconds")

This is the start of the timer...
Printed after 3 seconds


### Creating Portfolio Total Table of Subaccounts

In [36]:
# Creating a copy of the append table to total the dollar_amount column. Then I'll split off another process that'll create the total portfolio table.

In [ ]:
# Authentication in the headers. Defining the api keys with their own variables:
api_key = env.kc_api_key
api_s = env.kc_secret_api
api_pp = env.kc_passphrase
api_uid = env.kc_uid

# creating the api keys for use in the calls:
api_key = env.kc_api_key
api_secret = env.kc_secret_api
api_passphrase = env.kc_passphrase
url = 'https://api.kucoin.com/api/v1/sub-accounts'
now = int(time.time() * 1000)
str_to_sign = str(now) + 'GET' + '/api/v1/sub-accounts'
signature = base64.b64encode(
hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
headers = {
"KC-API-SIGN": signature,
"KC-API-TIMESTAMP": str(now),
"KC-API-KEY": api_key,
"KC-API-PASSPHRASE": passphrase,
"KC-API-KEY-VERSION": str(2)
}


# Getting the base response with the top level account values:
response_subaccts = requests.request('get', url, headers=headers)


# I haven't created any sub-accounts in KuCoin yet, which is why the list is empty.
response_subaccts.json()

-----------------------------

## Crontab Code for Automatically running the script

- I will eventually load this up to AWS so that the script and database are stored online, but for now this is the process I'm using to learn.

In [4]:
df = kucoin_append()
df

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-12 19:08:51,USDC,trade,1879.263101,1879.263101,0.00000,NaN,1879.26
2021-09-12 19:08:51,USDT,trade,1793.670859,1334.388009,459.28285,NaN,1793.67
2021-09-12 19:08:51,BTC3L,trade,918.961600,488.156400,430.80520,0.7877,723.87
2021-09-12 19:08:51,BTC,trade,0.013654,0.013654,0.00000,45894.3000,626.66
2021-09-12 19:08:51,BTC3S,trade,225.609100,166.834800,58.77430,2.2630,510.55
2021-09-12 19:08:51,ATOM,trade,0.236600,0.236600,0.00000,32.9815,7.80
2021-09-12 19:08:51,ALGO,trade,0.000000,0.000000,0.00000,2.3682,0.00
2021-09-12 19:08:51,ETH,trade,0.000000,0.000000,0.00000,3408.6600,0.00
2021-09-12 19:08:51,MATIC,trade,0.000000,0.000000,0.00000,1.3390,0.00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9 entries, 2021-09-12 19:08:51 to 2021-09-12 19:08:51
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   currency      9 non-null      object 
 1   act_name      9 non-null      object 
 2   balance       9 non-null      float64
 3   available     9 non-null      float64
 4   holds         9 non-null      float64
 5   price         7 non-null      float64
 6   dollar_value  9 non-null      float64
dtypes: float64(5), object(2)
memory usage: 576.0+ bytes


In [8]:
total_port = df.dollar_value.sum(axis = 0, skipna = True)

In [14]:
df.dollar_value.iloc[1] // total_port

0.0

In [17]:
5/2

2.5

In [9]:
display(df)

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-12 19:08:51,USDC,trade,1879.263101,1879.263101,0.00000,NaN,1879.26
2021-09-12 19:08:51,USDT,trade,1793.670859,1334.388009,459.28285,NaN,1793.67
2021-09-12 19:08:51,BTC3L,trade,918.961600,488.156400,430.80520,0.7877,723.87
2021-09-12 19:08:51,BTC,trade,0.013654,0.013654,0.00000,45894.3000,626.66
2021-09-12 19:08:51,BTC3S,trade,225.609100,166.834800,58.77430,2.2630,510.55
2021-09-12 19:08:51,ATOM,trade,0.236600,0.236600,0.00000,32.9815,7.80
2021-09-12 19:08:51,ALGO,trade,0.000000,0.000000,0.00000,2.3682,0.00
2021-09-12 19:08:51,ETH,trade,0.000000,0.000000,0.00000,3408.6600,0.00
2021-09-12 19:08:51,MATIC,trade,0.000000,0.000000,0.00000,1.3390,0.00


In [12]:
coin_lists = df.dollar_value.to_list()

In [29]:
coin_percentages = []

for i in coin_lists:
    coin_percentages.append(round(i / total_port, 2))

In [30]:
coin_lists, coin_percentages

([1879.26, 1793.67, 723.87, 626.66, 510.55, 7.8, 0.0, 0.0, 0.0],
 [0.34, 0.32, 0.13, 0.11, 0.09, 0.0, 0.0, 0.0, 0.0])

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9 entries, 2021-09-12 19:08:51 to 2021-09-12 19:08:51
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   currency      9 non-null      object 
 1   act_name      9 non-null      object 
 2   balance       9 non-null      float64
 3   available     9 non-null      float64
 4   holds         9 non-null      float64
 5   price         7 non-null      float64
 6   dollar_value  9 non-null      float64
dtypes: float64(5), object(2)
memory usage: 576.0+ bytes


In [32]:
# Cdf

,currency,act_name,balance,available,holds,price,dollar_value
date,,,,,,,
2021-09-12 19:08:51,USDC,trade,1879.263101,1879.263101,0.00000,NaN,1879.26
2021-09-12 19:08:51,USDT,trade,1793.670859,1334.388009,459.28285,NaN,1793.67
2021-09-12 19:08:51,BTC3L,trade,918.961600,488.156400,430.80520,0.7877,723.87
2021-09-12 19:08:51,BTC,trade,0.013654,0.013654,0.00000,45894.3000,626.66
2021-09-12 19:08:51,BTC3S,trade,225.609100,166.834800,58.77430,2.2630,510.55
2021-09-12 19:08:51,ATOM,trade,0.236600,0.236600,0.00000,32.9815,7.80
2021-09-12 19:08:51,ALGO,trade,0.000000,0.000000,0.00000,2.3682,0.00
2021-09-12 19:08:51,ETH,trade,0.000000,0.000000,0.00000,3408.6600,0.00
2021-09-12 19:08:51,MATIC,trade,0.000000,0.000000,0.00000,1.3390,0.00


In [43]:
# Creating the dataframe:

df = kucoin_append()

# Calcuating the total portfolio value in $:

total_port = df.dollar_value.sum(axis = 0, skipna = True)

# adding each holding's percentage of the overall portfolio:

df['percentages'] = round(df.dollar_value / total_port, 2)
df

,currency,act_name,balance,available,holds,price,dollar_value,percentages
date,,,,,,,,
2021-09-12 22:13:07,USDC,trade,1879.263101,1879.263101,0.000000,NaN,1879.26,0.34
2021-09-12 22:13:07,USDT,trade,1817.890668,1336.740966,481.149703,NaN,1817.89,0.33
2021-09-12 22:13:07,BTC3L,trade,951.422700,488.156400,463.266300,0.7687,731.36,0.13
2021-09-12 22:13:07,BTC,trade,0.013654,0.013654,0.000000,45380.6000,619.64,0.11
2021-09-12 22:13:07,BTC3S,trade,205.142800,166.834800,38.308000,2.3464,481.35,0.09
2021-09-12 22:13:07,ATOM,trade,0.236600,0.236600,0.000000,34.0659,8.06,0.00
2021-09-12 22:13:07,ALGO,trade,0.000000,0.000000,0.000000,2.3378,0.00,0.00
2021-09-12 22:13:07,ETH,trade,0.000000,0.000000,0.000000,3347.4500,0.00,0.00
2021-09-12 22:13:07,MATIC,trade,0.000000,0.000000,0.000000,1.3148,0.00,0.00


In [42]:
df

,currency,act_name,balance,available,holds,price,dollar_value,percentages
date,,,,,,,,
2021-09-12 19:08:51,USDC,trade,1879.263101,1879.263101,0.00000,NaN,1879.26,0.34
2021-09-12 19:08:51,USDT,trade,1793.670859,1334.388009,459.28285,NaN,1793.67,0.32
2021-09-12 19:08:51,BTC3L,trade,918.961600,488.156400,430.80520,0.7877,723.87,0.13
2021-09-12 19:08:51,BTC,trade,0.013654,0.013654,0.00000,45894.3000,626.66,0.11
2021-09-12 19:08:51,BTC3S,trade,225.609100,166.834800,58.77430,2.2630,510.55,0.09
2021-09-12 19:08:51,ATOM,trade,0.236600,0.236600,0.00000,32.9815,7.80,0.00
2021-09-12 19:08:51,ALGO,trade,0.000000,0.000000,0.00000,2.3682,0.00,0.00
2021-09-12 19:08:51,ETH,trade,0.000000,0.000000,0.00000,3408.6600,0.00,0.00
2021-09-12 19:08:51,MATIC,trade,0.000000,0.000000,0.00000,1.3390,0.00,0.00
